In [ ]:

from compare_categ import preprocess, read_csv, compare
from collections import defaultdict
import csv 
from csv import reader
from scipy.optimize import linear_sum_assignment
#from PIL import Image as im
from IPython.core.display import Image, display
from IPython.display import clear_output
import requests
from io import BytesIO
import json 
import pickle as pkl 


In [ ]:
def write(path, data):
    with open(path, 'wb') as f:
        pkl.dump(data, f)

## Round 1:

In [ ]:
data = read_csv("categ/csv_clean.csv")



annotated = []
annotated_path = "categ/purpose_goal_annotated.pkl"
for row in data:
    reason, r1, r2= json.loads(row['Answer.skip_reason'])

    if "reason" in reason or "~" in reason or "A/D" in reason:
        groups = []
        new_questions = json.loads(row['Answer.answer_questions'])
        for i, g in enumerate(json.loads(row['Answer.answer_groups'])):
            group = []
            new_question = new_questions[i]
            for x in g:
                group.append(x['content'])
            groups.append({new_question: group})
        print(f"question: {row['Input.questionStr']}")
        print(f"groups: {groups}")
        print(f"reason: {reason}")
        image_url = row['Input.imgUrl'].strip('"')
        display(Image(url=image_url))
        ans = input("New class: A/P, A/G, A/C, skip?")
        ans = ans.strip()
        row['Answer.skip_reason'] = json.dumps([ans, r1, r2])
        annotated.append(row)
        write(annotated_path, annotated)
     # else:
    #     clear_output(wait=True)

In [ ]:

# display only
with open("categ/purpose_goal_annotated.pkl", "rb") as f:
    data = pkl.load(f)


for row in data: 
    reason, r1, r2= json.loads(row['Answer.skip_reason'])

    groups = []
    new_questions = json.loads(row['Answer.answer_questions'])
    for i, g in enumerate(json.loads(row['Answer.answer_groups'])):
        group = []
        new_question = new_questions[i]
        for x in g:
            group.append(x['content'])
        groups.append({new_question: group})
    print(f"question: {row['Input.questionStr']}")
    print(f"groups: {groups}")
    print(f"reason: {reason}")
    image_url = row['Input.imgUrl'].strip('"')
    display(Image(url=image_url))
    ans = input("Continue?") 
    clear_output(wait=True)


## Round 2 

In [ ]:

with open("categ/purpose_goal_annotated.pkl", "rb") as f:
    data = pkl.load(f)


annotated = []
annotated_path = "categ/purpose_goal_annotated_round_2.pkl"
for row in data:
    reason, r1, r2= json.loads(row['Answer.skip_reason'])

    if "skip" in reason or "~" in reason:
        continue
    else:
        # A/G: goal
        # A/P: purpose
        # A/C: cause 
        # A/N: manner 
        groups = []
        new_questions = json.loads(row['Answer.answer_questions'])
        for i, g in enumerate(json.loads(row['Answer.answer_groups'])):
            group = []
            new_question = new_questions[i]
            for x in g:
                group.append(x['content'])
            groups.append({new_question: group})
        print(f"question: {row['Input.questionStr']}")
        print(f"groups: {groups}")
        print(f"reason: {reason}")
        image_url = row['Input.imgUrl'].strip('"')
        display(Image(url=image_url))
        ans = input("New class: A/P, A/G, A/C, A/N skip?")
        ans = ans.strip()
        row['Answer.skip_reason'] = json.dumps([ans, r1, r2])
        annotated.append(row)
        write(annotated_path, annotated)
     # else:
    #     clear_output(wait=True)

In [9]:
# merge and save 
with open("categ/purpose_goal_annotated.pkl", "rb") as f:
    data_round_1 = pkl.load(f)
with open("categ/purpose_goal_annotated_round_2.pkl", "rb") as f:
    data_round_2 = pkl.load(f)

round_2_keys = [row['Input.question_id'] for row in data_round_2]
combined = []

for row in data_round_1:
    if row['Input.question_id'] in round_2_keys:
        continue
    combined.append(row)
for row in data_round_2:
    combined.append(row)

csv_path = "categ/purpose_goal_annotated.csv"
with open(csv_path, "w") as f1:
    writer = csv.DictWriter(f1, fieldnames=combined[0].keys())
    writer.writeheader()
    writer.writerows(combined)